## Week 5 Homework 

In this homework we'll put what we learned about Spark in practice.

For this homework we will be using the FHV 2019-10 data found here. [FHV Data](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz)

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/06 05:23:44 WARN Utils: Your hostname, queen resolves to a loopback address: 127.0.1.1; using 192.168.1.134 instead (on interface wlo1)
24/03/06 05:23:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/06 05:23:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Question 1
**Install Spark and PySpark** 

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

> [!NOTE]
> To install PySpark follow this [guide](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/05-batch/setup/pyspark.md)

In [3]:
print(spark.version)

3.3.2


### Question 2

**FHV October 2019**

Read the October 2019 FHV into a Spark Dataframe with a schema as we did in the lessons.

Repartition the Dataframe to 6 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 1MB
- 6MB
- 25MB
- 87MB


In [4]:
!wc -l fhv_tripdata_2019-10.csv

1897494 fhv_tripdata_2019-10.csv


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')

In [6]:
df.head()

Row(dispatching_base_num='B00009', pickup_datetime='2019-10-01 00:23:00', dropOff_datetime='2019-10-01 00:35:00', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00009')

In [7]:
import pandas as pd


In [8]:
!head -n 1001 fhv_tripdata_2019-10.csv > head.csv

In [14]:
df_pandas = pd.read_csv('head.csv')

In [15]:
df_pandas.iteritems = df_pandas.items

In [11]:
spark.createDataFrame(df_pandas).schema

TypeError: field Affiliated_base_number: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

In [18]:
df_pandas['Affiliated_base_number'] = df_pandas['Affiliated_base_number'].fillna("")

In [19]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [20]:
from pyspark.sql import types

In [21]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.LongType(), True), 
    types.StructField('DOlocationID', types.LongType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)]
)

In [24]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [25]:
df.repartition(6).write.parquet('./data')

In [26]:
!ls -lah ./data

total 39M
drwxr-xr-x 2 pinku pinku 4.0K Mar  6 06:22 .
drwxrwxr-x 4 pinku pinku 4.0K Mar  6 06:23 ..
-rw-r--r-- 1 pinku pinku 6.4M Mar  6 06:22 part-00000-d10de12e-114c-4ad3-9b70-7b13ce852549-c000.snappy.parquet
-rw-r--r-- 1 pinku pinku  51K Mar  6 06:22 .part-00000-d10de12e-114c-4ad3-9b70-7b13ce852549-c000.snappy.parquet.crc
-rw-r--r-- 1 pinku pinku 6.4M Mar  6 06:22 part-00001-d10de12e-114c-4ad3-9b70-7b13ce852549-c000.snappy.parquet
-rw-r--r-- 1 pinku pinku  51K Mar  6 06:22 .part-00001-d10de12e-114c-4ad3-9b70-7b13ce852549-c000.snappy.parquet.crc
-rw-r--r-- 1 pinku pinku 6.4M Mar  6 06:22 part-00002-d10de12e-114c-4ad3-9b70-7b13ce852549-c000.snappy.parquet
-rw-r--r-- 1 pinku pinku  51K Mar  6 06:22 .part-00002-d10de12e-114c-4ad3-9b70-7b13ce852549-c000.snappy.parquet.crc
-rw-r--r-- 1 pinku pinku 6.4M Mar  6 06:22 part-00003-d10de12e-114c-4ad3-9b70-7b13ce852549-c000.snappy.parquet
-rw-r--r-- 1 pinku pinku  51K Mar  6 06:22 .part-00003-d10de12e-114c-4ad3-9b70-7b13ce852549-c000.snappy.par

### Question 3: 

**Count records** 

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

- 108,164
- 12,856
- 452,470
- 62,610

> [!IMPORTANT]
> Be aware of columns order when defining schema


In [28]:
df.createOrReplaceTempView("FHV_Records")

In [29]:
spark.sql("""
SELECT count(*)
FROM FHV_Records
WHERE pickup_datetime >= '2019-10-15'
    AND pickup_datetime < '2019-10-16'
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



### Question 4: 

**Longest trip for each day** 

What is the length of the longest trip in the dataset in hours?

- 631,152.50 Hours
- 243.44 Hours
- 7.68 Hours
- 3.32 Hours


In [30]:
spark.sql("""
SELECT pickup_datetime
    , dropoff_datetime
    , (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) / 3600  as tripdiff
FROM FHV_Records
ORDER BY tripdiff desc 
LIMIT 10
""").show()

+-------------------+-------------------+------------------+
|    pickup_datetime|   dropoff_datetime|          tripdiff|
+-------------------+-------------------+------------------+
|2019-10-11 18:00:00|2091-10-11 18:30:00|          631152.5|
|2019-10-28 09:00:00|2091-10-28 09:30:00|          631152.5|
|2019-10-31 23:46:33|2029-11-01 00:13:00| 87672.44083333333|
|2019-10-01 21:43:42|2027-10-01 21:45:23| 70128.02805555555|
|2019-10-17 14:00:00|2020-10-18 00:00:00|            8794.0|
|2019-10-26 21:26:00|2020-10-26 21:36:00| 8784.166666666666|
|2019-10-30 12:30:04|2019-12-30 13:02:08|1465.5344444444445|
|2019-10-25 07:04:57|2019-12-08 07:54:33|1057.8266666666666|
|2019-10-25 07:04:57|2019-12-08 07:21:11|1057.2705555555556|
|2019-10-01 13:47:17|2019-11-03 15:20:28| 794.5530555555556|
+-------------------+-------------------+------------------+





### Question 5: 

**User Interface**

Spark’s User Interface which shows the application's dashboard runs on which local port?

- 80
- 443
- 4040
- 8080

### Question 6: 

**Least frequent pickup location zone**

Load the zone lookup data into a temp view in Spark</br>
[Zone Data](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv)

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?</br>

- East Chelsea
- Jamaica Bay
- Union Sq
- Crown Heights North

In [31]:
zone_df = spark.read.csv("taxi_zone_lookup.csv", header=True)

In [33]:
zone_df.createOrReplaceTempView("zones")

In [34]:
zone_df.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [36]:
spark.sql("""
SELECT * 
FROM zones
LIMIT 5""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [39]:
spark.sql("""
SELECT * 
FROM fhv_records
LIMIT 5""").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   null|                B00014|
+--------------------+------------------

In [37]:
spark.sql("""
SHOW VIEWS;
""").show()

+---------+-----------+-----------+
|namespace|   viewName|isTemporary|
+---------+-----------+-----------+
|         |fhv_records|       true|
|         |      zones|       true|
+---------+-----------+-----------+



In [38]:
spark.sql("""
SHOW TABLES;
""").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|         |fhv_records|       true|
|         |      zones|       true|
+---------+-----------+-----------+



In [42]:
spark.sql("""
SELECT PULocationID
    , zone
    , COUNT(*) as cnt
FROM fhv_records 
    INNER JOIN zones
        ON PULocationID = LocationID
GROUP BY PULocationID, zone
ORDER BY cnt ASC
""").show()

+------------+--------------------+---+
|PULocationID|                zone|cnt|
+------------+--------------------+---+
|           2|         Jamaica Bay|  1|
|         105|Governor's Island...|  2|
|         111| Green-Wood Cemetery|  5|
|          30|       Broad Channel|  8|
|         120|     Highbridge Park| 14|
|          12|        Battery Park| 15|
|         207|Saint Michaels Ce...| 23|
|          27|Breezy Point/Fort...| 25|
|         154|Marine Park/Floyd...| 26|
|           8|        Astoria Park| 29|
|         128|    Inwood Hill Park| 39|
|         253|       Willets Point| 47|
|          96|Forest Park/Highl...| 53|
|          34|  Brooklyn Navy Yard| 57|
|          59|        Crotona Park| 62|
|          58|        Country Club| 77|
|          99|     Freshkills Park| 89|
|         190|       Prospect Park| 98|
|          54|     Columbia Street|105|
|         217|  South Williamsburg|110|
+------------+--------------------+---+
only showing top 20 rows



In [41]:
zone_df.show(270)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly